In [13]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    60% |███████████████████▍            | 358.2MB 8.5MB/s eta 0:00:28

    100% |████████████████████████████████| 592.3MB 12.7MB/s 


In [14]:
!pip3 install torchvision

    100% |████████████████████████████████| 51kB 1.7MB/s 
    100% |████████████████████████████████| 5.9MB 211kB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


In [5]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving protein_amino_acid_mapping.txt to protein_amino_acid_mapping.txt
Saving train_file.txt to train_file (1).txt
User uploaded file "protein_amino_acid_mapping.txt" with length 25601 bytes
User uploaded file "train_file.txt" with length 436 bytes


In [0]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
import random
from torch.utils.data import DataLoader,Dataset
import numpy as np

In [0]:
class Config():
  train_batch_size = 32
  train_number_epochs = 2
  learning_rate = .0005

In [0]:
char_to_index = {'M': 0, 'S': 1, 'V': 2, 'E': 3, 'D': 4, 'F': 5, 'I': 6, 'Q': 7, 'P': 8, 'Y': 9, 'T': 10, 'L': 11, 'N': 12, 'R': 13, 'G': 14, 'K': 15, 'C': 16, 'H': 17, 'A': 18, 'W': 19, 'U': 20}
index_to_char = {0: 'M', 1: 'S', 2: 'V', 3: 'E', 4: 'D', 5: 'F', 6: 'I', 7: 'Q', 8: 'P', 9: 'Y', 10: 'T', 11: 'L', 12: 'N', 13: 'R', 14: 'G', 15: 'K', 16: 'C', 17: 'H', 18: 'A', 19: 'W', 20: 'U'}
n_letters = len(char_to_index)

In [0]:
#https://colab.research.google.com/notebook#fileId=/v2/external/notebooks/io.ipynb&scrollTo=p2E4EKhCWEC5

In [0]:
def letterToIndex(letter):
  return char_to_index[letter]

def letterToTensor(letter):
  tensor = torch.zeros(1, n_letters)
  tensor[0][letterToIndex(letter)] = 1
  return tensor

def aminoAcidstoTensor(aminoAcidSeq):
  tensor = torch.zeros(len(aminoAcidSeq), 1, n_letters)
  for idx, letter in enumerate(aminoAcidSeq):
    tensor[idx][0][letterToIndex(letter)] = 1
  return tensor   
  

In [0]:
categories = [0, 1]
proteins = {}
training_data = []


In [0]:
with open("protein_amino_acid_mapping.txt") as protein_data:
  for line in protein_data:
    line = line.strip('\n')
    name_seq_pair = line.split(' ')
    proteins[name_seq_pair[0]] = name_seq_pair[1]
                                         
protein_data.close()

with open('train_file.txt') as train_data:
  for line in train_data:
    line = line.strip('\n')
    training_data.append(line)
    
train_data.close()

In [0]:
class ProteinDataset(Dataset):
  def __init__(self, data):
    self.data = data
    self.count = 0
    print(self.data)
    
  
  def __getitem__(self, index):
    sample = self.data[self.count]
    pair_label = sample.split(' ')
    protein_pair = pair_label[0].split('-')
    protein_name_1 = protein_pair[0]
    protein_name_2 = protein_pair[1]
    label = pair_label[1]
    
    protein_seq_1 = proteins[protein_name_1]
    protein_seq_2 = proteins[protein_name_2]
    
    #Variable(aminoAcidstoTensor(protein_seq_1)).cuda()
    protein_1_tensor = Variable(aminoAcidstoTensor(protein_seq_1))
    protein_2_tensor = Variable(aminoAcidstoTensor(protein_seq_2))
    
    #torch.cuda.LongTensor?
    label_tensor = Variable(torch.LongTensor([categories.index(0)]))
    self.count += 1
    
    #demensions returned value: len of amino acid seq x 1 x #amino acids (21)
    return protein_1_tensor, protein_2_tensor, label_tensor
  
  def __len__():
    return len(self.data)
    
    

In [0]:
protein_dataset = ProteinDataset(training_data)

protein_dataset.__getitem__(0)
#[categories.index(0)]


In [0]:
train_dataloader = DataLoader(protein_dataset, shuffle=True, num_workers=1, batch_size=Config.train_batch_size)

In [0]:
class Net(nn.Module):
    def __init__(self, hidden_size):
        super(Net, self).__init__()
        #could also pass in num_layers, but assume 1 in this case
        #inchannels=input height, outchannels=number filters, kernel size= filter size
        self.conv1 = nn.Conv1d(21, 10, 5)
        self.pool1 = nn.MaxPool1d(2, 2)
        
        #self.lstm = nn.LSTM(input_size, hidden_size, bidirectional=True)
        self.lstm = nn.LSTM(hidden_size, hidden_size, bidirectional=True)
        #self.fc = nn.Linear(hidden_size * 2, num_classes)
        self.fc = nn.Linear(hidden_size * 2, 2)


    def forward_once(self, x):
        h0 = Variable(torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size))  # 2 for bidirection
        c0 = Variable(torch.zeros(self.num_layers * 2, x.size(0), self.hidden_size))

        x = self.pool1(F.relu(self.conv1(x)))
        out, (h_n, c_n) = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])

        return x


    def forward(self, x1, x2):
        output1 = self.forward_once(x1)
        output2 = self.forward_once(x2)
        return output1, output2

In [0]:
class Net_Test(nn.Module):
    def __init__(self):
        super(Net_Test, self).__init__()
        #could also pass in num_layers, but assume 1 in this case
        #inchannels=input height, outchannels=number filters, kernel size= filter size
        self.conv1 = nn.Conv1d(1, 6, 5)
        self.pool1 = nn.MaxPool1d(2, 2)
        


    def forward(self, x):
        print("before dim", x.dim())
        x = F.relu(self.conv1(x))
        print("after dim", x.dim())
        return x



In [0]:
def printnorm(self, input, output):
    # input is a tuple of packed inputs
    # output is a Variable. output.data is the Tensor we are interested
    print('Inside ' + self.__class__.__name__ + ' forward')
    print('')
    print('input: ', type(input))
    print('input[0]: ', type(input[0]))
    print('output: ', type(output))
    print('')
    print('input size:', input[0].size())
    print('output size:', output.data.size())
    print('output norm:', output.data.norm())


def printgradnorm(self, grad_input, grad_output):
    print('Inside ' + self.__class__.__name__ + ' backward')
    print('Inside class:' + self.__class__.__name__)
    print('')
    print('grad_input: ', type(grad_input))
    print('grad_input[0]: ', type(grad_input[0]))
    print('grad_output: ', type(grad_output))
    print('grad_output[0]: ', type(grad_output[0]))
    print('')
    print('grad_input size:', grad_input[0].size())
    print('grad_output size:', grad_output[0].size())
    print('grad_input norm:', grad_input[0].data.norm())

In [0]:
net = Net_Test()
print(net)

net.conv1.register_forward_hook(printnorm)
net.conv1.register_backward_hook(printgradnorm)

input = Variable(torch.randn(1, 32, 32))
out = net(input)
print(out)

In [0]:
class ContrastiveLoss(torch.nn.Module):
    """
    Contrastive loss function.
    Based on: http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    """

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean((1-label) * torch.pow(euclidean_distance, 2) +
                                      (label) * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2))


        return loss_contrastive

In [140]:
net = Net(1)
print(net)

Net(
  (conv1): Conv1d (21, 10, kernel_size=(5,), stride=(1,))
  (pool1): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (lstm): LSTM(1, 1, bidirectional=True)
  (fc): Linear(in_features=2, out_features=2)
)


In [110]:
#Net().cuda
net = Net(1)
print(net)
net.conv1.register_forward_hook(printnorm)
net.conv1.register_backward_hook(printgradnorm)

out = net(input)
criterion = ContrastiveLoss()
optimizer = optim.Adam(net.parameters(), Config.learning_rate)

counter = []
loss_history = []
iteration_number = 0

for epoch in range(Config.train_number_epochs):
    for i, data in enumerate(train_dataloader):
        protein1, protein2, label = data
        output1, output2 = net(protein1, protein2)
        optimizer.zero_grad()
        contrastive_loss = criterion(output1, output2, label)
        contrastive_loss.backward()
        optimizer.step()

TypeError: ignored